## Imports

In [ ]:
%load_ext lab_black

import h5py
import os
import numpy as np
from typing import Dict, List, Optional, Tuple

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache


import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    kw_data_loader,
    cnn,
    polygons,
)

# Load Data

In [ ]:
## Load averaged, K corrected data ##

ddir = r"E:\atully\arpes_data\2023_February\6eV\FS"
files = ["FS1_FS3_Ali_avg_gkw.h5"]

# This works, but makes dataclass with theta and phi_or_time instead of kx and ky
ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

In [ ]:
ad_right = ARPES_DATA[files[0]]
for k in ["energy", "theta", "phi_or_time"]:
    print(f"{k}.shape = {getattr(ad_right, k).shape}")
print(f"Data.shape = {ad_right.data.shape}")

In [ ]:
## Load K corrected data ##

ddir = r"E:\atully\arpes_data\2023_February\6eV\FS"
files = ["FS8_avg_gkw.h5"]

# This works, but makes dataclass with theta and phi_or_time instead of kx and ky
ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

In [ ]:
ad_left = ARPES_DATA[files[0]]
for k in ["energy", "theta", "phi_or_time"]:
    print(f"{k}.shape = {getattr(ad_left, k).shape}")
print(f"Data.shape = {ad_left.data.shape}")

# Analysis -- stitching and normalizing datasets

In [ ]:
## Initial params ##
slice_dim = "y"
slice_val = 2.15
# slice_val = 2.05
# slice_val = 1.95
int_range = 0.05

In [ ]:
## Slice Data ##

## Initial params
xlim = (-0.12, 0.45)  # for k-corrected data
# xlim = None
ylim = (-0.52, 0.17)  # for k-corrected data
x_bin = 2
y_bin = 2

## Get data
x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad_right,
    slice_dim,
    slice_val,
    int_range,
    xlim,
    ylim,
    x_bin,
    y_bin,
    norm_data=True,
    plot_data=False,
)

## Plot data
fig = tr_functions.default_fig()
fig.add_trace(go.Heatmap(x=x_2d, y=y_2d, z=d_2d, coloraxis="coloraxis"))
# fig.update_coloraxes(colorscale="Blues", showscale=True)
# fig.update_coloraxes(colorscale="dense", showscale=True)
fig.update_coloraxes(colorscale="ice", reversescale=True, showscale=True)
fig.update_layout(
    title=dict(text=f"2D slice", x=0.5, xanchor="center", yanchor="top", font_size=22),
    xaxis_title="$k_x \; [A^{-1}]$",  # or \quad for larger space
    yaxis_title="$k_y \; [A^{-1}]$",
)
fig.update_layout(height=500, width=500)

fig.show(renderer="svg")

x_right, y_right, data_right = x_2d, y_2d, d_2d

In [ ]:
## Slice Data ##

## Initial params
ylim = (-0.52, 0.16)  # same as above
xlim = (-0.57, -0.04)  # for k-corrected data
# ylim = (-0.44, 0.2)  # for k-corrected data
x_bin = 2
y_bin = 2

## Get data
x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad_left,
    slice_dim,
    slice_val,
    int_range,
    xlim,
    ylim,
    x_bin,
    y_bin,
    norm_data=True,
    plot_data=False,
)

## Plot data
fig = tr_functions.default_fig()
fig.add_trace(go.Heatmap(x=x_2d, y=y_2d, z=d_2d, coloraxis="coloraxis"))
# fig.update_coloraxes(colorscale="Blues", showscale=True)
# fig.update_coloraxes(colorscale="dense", showscale=True)
fig.update_coloraxes(colorscale="ice", reversescale=True, showscale=True)
fig.update_layout(
    title=dict(text=f"2D slice", x=0.5, xanchor="center", yanchor="top", font_size=22),
    xaxis_title="$k_x \; [A^{-1}]$",  # or \quad for larger space
    yaxis_title="$k_y \; [A^{-1}]$",
)
fig.update_layout(height=500, width=500)

fig.show(renderer="svg")

x_left, y_left, data_left = x_2d, y_2d, d_2d

In [ ]:
## Normalize plots relative to backgrounds ##

# norm_to_bg = True
norm_to_bg = False

if norm_to_bg:
    norm_type = "background"
else:
    norm_type = "peak"

# ad_right
# normalize to backgrounds
if norm_type == "background":
    xlim = (0.369, 0.418)
    ylim = (-0.492, -0.452)
# normalize to peaks rather than backgrounds
elif norm_type == "peak":
    xlim = (0.340, 0.373)
    ylim = (-0.153, -0.112)

ad_right_bg = tr_functions.get_avg_background(x_right, y_right, data_right, xlim, ylim)

# ad_left
# normalize to backgrounds
if norm_type == "background":
    xlim = (-0.512, -0.403)
    ylim = (-0.453, -0.326)
# normalize to peaks rather than backgrounds
elif norm_type == "peak":
    xlim = (-0.359, 0.289)
    ylim = (0.056, 0.112)


ad_left_bg = tr_functions.get_avg_background(x_left, y_left, data_left, xlim, ylim)
print(ad_left_bg, ad_right_bg)

norm_right_to_left = ad_left_bg / ad_right_bg
data_right_norm = data_right * norm_right_to_left

In [ ]:
## Check resolutions match where we will be stitching ##

x_left.shape, y_left.shape, data_left.shape, x_right.shape, y_right.shape, data_right_norm.shape

In [ ]:
## Plot Data with Basic Stitch (not really stitched) ##

from scipy.interpolate import interp1d

x1, x2, dataslice1, dataslice2 = x_right, x_left, data_right, data_left

new_x = np.linspace(min(min(x1), min(x2)), max(max(x1), max(x2)), len(x1) + len(x2))

new_datas = []
for x, data in zip([x1, x2], [dataslice1, dataslice2]):
    nd = []
    for row in data:
        interper = interp1d(x, row, fill_value=np.nan, bounds_error=False)
        nd.append(interper(new_x))
    new_datas.append(np.array(nd))

fig = tr_functions.thesis_fig(
    title=f"Excited State: {slice_val} eV",
    xaxis_title="$k_x \; [A^{-1}]$",
    yaxis_title="$k_y \; [A^{-1}]$",
)

fig.add_trace(
    go.Heatmap(
        x=new_x,
        y=y_left,
        z=new_datas[0],
        coloraxis="coloraxis",
    )
)
fig.add_trace(
    go.Heatmap(
        x=new_x,
        y=y_left,
        z=new_datas[1],
        coloraxis="coloraxis",
    )
)

fig.update_coloraxes(
    colorscale="ice",
    reversescale=True,
    showscale=True,
    cmin=0,
    cmax=None,
)
# fig.update_layout(width=800, height=600)
fig.show(renderer="svg")

# fig.write_image(
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_2.15eV_plainstitch.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_2.05eV_plainstitch.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_1.95eV_plainstitch.png"
# )

In [ ]:
## Stitch normalized data together ##

x_s1, y_s1, data_s1 = tr_functions.stitch_2_datasets(
    data_left, x_left, y_left, data_right_norm, x_right, y_right, stitch_dim="x"
)

In [ ]:
## Plot Data ##

fig = tr_functions.thesis_fig(
    title=f"Excited State: {slice_val} eV",
    xaxis_title="$k_x \; [A^{-1}]$",
    yaxis_title="$k_y \; [A^{-1}]$",
)

fig.add_trace(
    go.Heatmap(
        x=x_s1, y=y_s1, z=analysis_functions.norm_data(data_s1), coloraxis="coloraxis"
    )
)

# hexagon = polygons.gen_polygon(6, 0.42, rotation=30)
# fig = polygons.plot_polygon(hexagon, color="green", fig=fig, show=False)

fig.update_coloraxes(
    colorscale="ice",
    reversescale=True,
    showscale=True,
    cmin=0,
    cmax=None,
)
# fig.update_layout(width=800, height=600)
fig.show(renderer="svg")
# fig.show()

# fig.write_image(
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_2.15eV_normpeaks.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_2.05eV_normpeaks.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_1.95eV_normpeaks.png"
# )
# fig.write_image(
#     r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_2.05eV_normbg.png"
# )

In [ ]:
## Rotate Data ##

x, y, z = x_s1, y_s1, analysis_functions.norm_data(data_s1)

coords = tr_functions.x_y_to_coords(x, y)

rotated_coords = tr_functions.rotate_2d_array(coords, 120, (0, 0))
rotated_coords_2 = tr_functions.rotate_2d_array(coords, 240, (0, 0))

nx, ny, nd = tr_functions.interpolate(rotated_coords, z)
nx_2, ny_2, nd_2 = tr_functions.interpolate(rotated_coords_2, z)

In [ ]:
## Average original dataset with 1st rotated dataset ##

x1, y1, dataslice1 = x_s1, y_s1, analysis_functions.norm_data(data_s1)
x2, y2, dataslice2 = nx, ny, nd

new_x, new_y, new_data = tr_functions.stitch_and_avg(
    x1, y1, dataslice1, x2, y2, dataslice2
)

In [ ]:
## Average new dataset with 2nd rotated dataset ##

x1, y1, dataslice1 = new_x, new_y, new_data
x2, y2, dataslice2 = nx_2, ny_2, nd_2

new_x_stitched, new_y_stitched, new_data_stitched = tr_functions.stitch_and_avg(
    x1, y1, dataslice1, x2, y2, dataslice2
)

In [ ]:
# Plot raw data and rotations on same figure; plot averaged data ##

fig = tr_functions.thesis_fig(
    title=f"Excited State: {slice_val} eV",
    xaxis_title="$k_x \; [A^{-1}]$",
    yaxis_title="$k_y \; [A^{-1}]$",
)


## Rotated Data
# fig.add_trace(
#     go.Heatmap(
#         x=nx,
#         y=ny,
#         z=nd,
#         coloraxis="coloraxis",
#     )
# )

# fig.add_trace(
#     go.Heatmap(
#         x=nx_2,
#         y=ny_2,
#         z=nd_2,
#         coloraxis="coloraxis",
#     )
# )

# fig.add_trace(
#     go.Heatmap(
#         x=x_s1,
#         y=y_s1,
#         z=analysis_functions.norm_data(data_s1),
#         coloraxis="coloraxis",
#     )
# )


## Averaged Data
fig.add_trace(
    go.Heatmap(
        x=new_x_stitched,
        y=new_y_stitched,
        z=analysis_functions.norm_data(new_data_stitched),
        coloraxis="coloraxis",
    )
)


hexagon = polygons.gen_polygon(6, 0.42, rotation=30)
fig = polygons.plot_polygon(
    hexagon, color="firebrick", fig=fig, show=False, dash=True, dash_width=3
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)

fig.update_layout(width=600, height=600)

fig.show(renderer="svg")

# fig.write_image(
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_2.15eV_FS138.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_2.05eV_FS138.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_1.95eV_FS138.png"
# )

# fig.write_image(
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_2.15eV_FS138_avg.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_2.05eV_FS138_avg.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_1.95eV_FS138_avg.png"
# )

## Attempt to reconstruct the BZ with only the right image

In [ ]:
## Slice Data ##

## Initial params
slice_dim = "y"
# slice_val = 2.15
slice_val = 2.05
# slice_val = 1.95
int_range = 0.05
xlim = (-0.12, 0.45)  # for k-corrected data
# xlim = None
ylim = (-0.52, 0.17)  # for k-corrected data
x_bin = 2
y_bin = 2

## Get data
x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad_right,
    slice_dim,
    slice_val,
    int_range,
    xlim,
    ylim,
    x_bin,
    y_bin,
    norm_data=True,
    plot_data=False,
)

In [ ]:
## Plot Data ##

fig = tr_functions.thesis_fig(
    title=f"Excited State: {slice_val} eV",
    xaxis_title="$k_x \; [A^{-1}]$",
    yaxis_title="$k_y \; [A^{-1}]$",
)

fig.add_trace(
    go.Heatmap(
        x=x_2d, y=y_2d, z=analysis_functions.norm_data(d_2d), coloraxis="coloraxis"
    )
)

hexagon = polygons.gen_polygon(6, 0.42, rotation=30)
fig = polygons.plot_polygon(
    hexagon, color="firebrick", fig=fig, show=False, dash=True, dash_width=3
)

fig.update_coloraxes(
    colorscale="ice",
    reversescale=True,
    showscale=True,
    cmin=0,
    cmax=None,
)

# fig.update_xaxes(range=[xlim[0], xlim[1]], constrain="domain")
fig.update_xaxes(range=[-0.45, xlim[1]], constrain="domain")
fig.update_yaxes(scaleanchor="x", scaleratio=1)

fig.update_layout(width=600, height=600)
fig.show()

# fig.write_image(
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_2.15eV_rightside_step1.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_2.05eV_rightside_step1.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_1.95eV_rightside_step1.png"
# )

In [ ]:
## This Results in the correct symmetry for K points, but not for M points!! Don't flip and symmeterize; instead just rotate and average ##

In [ ]:
# ## Flip across ky = 0 and stitch datasets together ##

# x, y, z = (
#     x_2d,
#     -1 * y_2d,
#     d_2d,
# )  # uses all the data reflected across ky=0 and stitches together

# x_s2, y_s2, data_s2 = tr_functions.stitch_2_datasets(
#     d_2d, x_2d, y_2d, z, x, y, stitch_dim="y"
# )

In [ ]:
# ## Plot Stitched Data ##

# fig = tr_functions.thesis_fig(
#     title=f"Excited State: {slice_val} eV",
#     xaxis_title="$k_x \; [A^{-1}]$",
#     yaxis_title="$k_y \; [A^{-1}]$",
# )

# fig.add_trace(
#     go.Heatmap(
#         x=x_s2, y=y_s2, z=analysis_functions.norm_data(data_s2), coloraxis="coloraxis"
#     )
# )

# hexagon = polygons.gen_polygon(6, 0.42, rotation=30)
# fig = polygons.plot_polygon(
#     hexagon, color="firebrick", fig=fig, show=False, dash=True, dash_width=3
# )

# fig.update_coloraxes(
#     colorscale="ice",
#     reversescale=True,
#     showscale=True,
#     cmin=0,
#     cmax=None,
# )

# # fig.update_xaxes(range=[xlim[0], xlim[1]], constrain="domain")
# fig.update_xaxes(range=[-0.45, xlim[1]], constrain="domain")
# fig.update_yaxes(scaleanchor="x", scaleratio=1)

# fig.update_layout(width=600, height=600)
# fig.show(renderer="svg")

# # fig.write_image(
# #     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_2.15eV_rightside_step2.png"
# #     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_2.05eV_rightside_step2.png"
# #     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_1.95eV_rightside_step2.png"
# # )

In [ ]:
## Rotate Data ##

In [ ]:
# x, y, z = x_s2, y_s2, analysis_functions.norm_data(data_s2)
x, y, z = x_2d, y_2d, analysis_functions.norm_data(d_2d)

coords = tr_functions.x_y_to_coords(x, y)

In [ ]:
## Increase resolution of original data to 1000x1000 ##

# rotated_coords_0 = tr_functions.rotate_2d_array(coords, 0, (0, 0))
# nx_0, ny_0, nd_0 = tr_functions.interpolate(rotated_coords_0, z)

In [ ]:
rotated_coords = tr_functions.rotate_2d_array(coords, 120, (0, 0))
rotated_coords_2 = tr_functions.rotate_2d_array(coords, 240, (0, 0))

In [ ]:
nx, ny, nd = tr_functions.interpolate(rotated_coords, z)
nx_2, ny_2, nd_2 = tr_functions.interpolate(rotated_coords_2, z)

In [ ]:
## Plot raw data and rotations on same figure ##

fig = tr_functions.thesis_fig(
    title=f"Excited State: {slice_val} eV",
    xaxis_title="$k_x \; [A^{-1}]$",
    yaxis_title="$k_y \; [A^{-1}]$",
)

fig.add_trace(
    go.Heatmap(
        x=nx,
        y=ny,
        z=nd,
        coloraxis="coloraxis",
    )
)

fig.add_trace(
    go.Heatmap(
        x=nx_2,
        y=ny_2,
        z=nd_2,
        coloraxis="coloraxis",
    )
)

## Looks the same when I increase resolution of original data to 1000x1000
# fig.add_trace(
#     go.Heatmap(
#         x=nx_0,
#         y=ny_0,
#         z=nd_0,
#         coloraxis="coloraxis",
#     )
# )

fig.add_trace(
    go.Heatmap(
        # x=x_s2, y=y_s2, z=analysis_functions.norm_data(data_s2), coloraxis="coloraxis"
        x=x_2d,
        y=y_2d,
        z=analysis_functions.norm_data(d_2d),
        coloraxis="coloraxis",
    )
)

hexagon = polygons.gen_polygon(6, 0.42, rotation=30)
fig = polygons.plot_polygon(
    hexagon, color="firebrick", fig=fig, show=False, dash=True, dash_width=3
)

fig.update_coloraxes(
    colorscale="ice",
    reversescale=True,
    showscale=True,
    cmin=0,
    cmax=None,
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)

fig.update_layout(width=600, height=600)

fig.show(renderer="svg")

# fig.write_image(
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_2.15eV_rightside_step2.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_2.05eV_rightside_step2.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_1.95eV_rightside_step2.png"
# )

In [ ]:
## Average original dataset with 1st rotated dataset ##

# x1, y1, dataslice1 = x_s2, y_s2, analysis_functions.norm_data(data_s2)
x1, y1, dataslice1 = x_2d, y_2d, analysis_functions.norm_data(d_2d)
x2, y2, dataslice2 = nx, ny, nd

new_x, new_y, new_data = tr_functions.stitch_and_avg(
    x1, y1, dataslice1, x2, y2, dataslice2
)

In [ ]:
## Average new dataset with 2nd rotated dataset ##

x1, y1, dataslice1 = new_x, new_y, new_data
x2, y2, dataslice2 = nx_2, ny_2, nd_2

new_x, new_y, new_data = tr_functions.stitch_and_avg(
    x1, y1, dataslice1, x2, y2, dataslice2
)


## Plot averaged data ##
fig = tr_functions.thesis_fig(
    title=f"Excited State: {slice_val} eV",
    xaxis_title="$k_x \; [A^{-1}]$",
    yaxis_title="$k_y \; [A^{-1}]$",
)

fig.add_trace(
    go.Heatmap(
        x=new_x,
        y=new_y,
        z=analysis_functions.norm_data(new_data),
        coloraxis="coloraxis",
    )
)

hexagon = polygons.gen_polygon(6, 0.42, rotation=30)
fig = polygons.plot_polygon(
    hexagon, color="firebrick", fig=fig, show=False, dash=True, dash_width=3
)

# fig.show()
fig.show(renderer="svg")


# fig.write_image(
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_2.15eV_full_averaged.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_2.05eV_full_averaged.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\FS_1.95eV_full_averaged.png"
# )